In [2]:
#This tutorial can be found on https://elitedatascience.com

##Goal: To introduce techniques for data wrangling in Python, which is the process of reshaping, aggregating, separating, or
#otherwise transforming your data from one format to a more useful one.

#For this tutorial, we'll be using a price dataset managed by Brave New Coin and distributed on
#Quandl (https://www.quandl.com/data/BNC2-BNC-Digital-Currency-Indexed-EOD). The full version
#tracks price indices for 1,900+ fiat-crypto trading pairs, but it requires a premium subscription, so
#we've provided a small sample with a handful of cryptocurrencies.

##Tasks: We wanted to run a step-forward analysis:

#1. At the start of every month, we buy the cryptocurrency that had the largest price gain over the previous 7, 14, 21, or 28 
#days. We want to evaluate each of these time windows.

#2. Then, we hold for exactly 7 days and sell our position.

##Summary:

#1. Set up your environment.
#2. Import libraries and dataset.
#3. Understand the data.
#4. Filter unwanted observations.
#5. Pivot the dataset.
#6. Shift the pivoted dataset.
#7. Melt the shifted dataset.
#8. Reduce-merge the melted data.
#9. Aggregate with group-by.

In [ ]:
#Step 1: Set up your environment.
#Here I'm using Python 3 in a Jupiter Notebook.

In [4]:
##Step 2: Import libraries and dataset.
import pandas as pd

#Display floats with two decimal places
pd.options.display.float_format = '{:,.2f}'.format

#Expand display limits
pd.options.display.max_rows = 200
pd.options.display.max_columns = 100

#Read the BNC2 sample dataset
df = pd.read_csv('./datasets/BNC2_sample.csv', 
                  names=['Code', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP', 'TWAP']) #Set cols names 
                                                                                                    #(missing in dataset)

#Display the first five observations:

df.head()

,Code,Date,Open,High,Low,Close,Volume,VWAP,TWAP
0,GWA_BTC,2014-04-01,467.28,488.62,467.28,479.56,"74,776.48",482.76,482.82
1,GWA_BTC,2014-04-02,479.20,494.30,431.32,437.08,"114,052.96",460.19,465.93
2,GWA_BTC,2014-04-03,437.33,449.74,414.41,445.60,"91,415.08",432.29,433.28
3,GWA_BTC,2014-04-04,445.18,456.10,429.16,449.81,"51,147.27",443.46,443.93
4,GWA_BTC,2014-04-05,450.08,464.09,445.16,461.70,"28,449.19",452.53,452.95


In [ ]:
##Data Dictionary (for code GWA_BTC):
#• Date: The day on which the index values were calculated.
#• Open: The day's opening price index for Bitcoin in US dollars.
#• High: The highest value for the price index for Bitcoin in US dollars that day.
#• Low: The lowest value for the price index for Bitcoin in US dollars that day.
#• Close: The day's closing price index for Bitcoin in US dollars.
#• Volume: The volume of Bitcoin traded that day.
#• VWAP: The volume weighted average price of Bitcoin traded that day.
#• TWAP: The time-weighted average price of Bitcoin traded that day.

In [5]:
##Step 3: Understand the data.

#Generally, all observations should be equivalent in granularity and in units.

#• Equivalence in Granularity - For example, you could have 10 rows of data from 10 different
#cryptocurrencies. However, you should not have an 11th row with average or total values from
#the other 10 rows. That 11th row would be an aggregation, and thus not equivalent in
#granularity to the other 10.

#• Equivalence in Units - You could have 10 rows with prices in USD collected at different dates.
#However, you should not then have another 10 rows with prices quoted in EUR. Any
#aggregations, distributions, visualizations, or statistics would become meaningless.

#The current raw dataset breaks both of these rules!

#For instance, watch at the unique codes in the dataset:

print(df.Code.unique())

['GWA_BTC' 'GWA_ETH' 'GWA_LTC' 'GWA_XLM' 'GWA_XRP' 'MWA_BTC_CNY'
 'MWA_BTC_EUR' 'MWA_BTC_GBP' 'MWA_BTC_JPY' 'MWA_BTC_USD' 'MWA_ETH_CNY'
 'MWA_ETH_EUR' 'MWA_ETH_GBP' 'MWA_ETH_JPY' 'MWA_ETH_USD' 'MWA_LTC_CNY'
 'MWA_LTC_EUR' 'MWA_LTC_GBP' 'MWA_LTC_JPY' 'MWA_LTC_USD' 'MWA_XLM_CNY'
 'MWA_XLM_EUR' 'MWA_XLM_USD' 'MWA_XRP_CNY' 'MWA_XRP_EUR' 'MWA_XRP_GBP'
 'MWA_XRP_JPY' 'MWA_XRP_USD']


In [9]:
#Some codes start with GWA and others with MWA? These are actually completely different types of indicators according to the 
#documentation page.

#• MWA stands for "market-weighted average," and they show regional prices. There are
#multiple MWA codes for each cryptocurrency, one for each local fiat currency.

#• On the other hand, GWA stands for "global-weighted average," which shows globally indexed
#prices. GWA is thus an aggregation of MWA and not equivalent in granularity. (Note: only a
#subset of regional MWA codes are included in the sample dataset.)

#let's look at Bitcoin's codes on the same date:

df[df.Code.isin(['GWA_BTC', 'MWA_BTC_JPY', 'MWA_BTC_EUR']) & (df.Date == '2018-01-01')]

,Code,Date,Open,High,Low,Close,Volume,VWAP,TWAP
1371,GWA_BTC,2018-01-01,"14,505.89","14,505.89","13,617.46","14,092.74","225,906.21","14,103.18","14,093.73"
9074,MWA_BTC_EUR,2018-01-01,"11,859.35","11,859.35","11,111.07","11,403.92","14,933.73","11,488.45","11,478.08"
11838,MWA_BTC_JPY,2018-01-01,"1,674,341.45","1,678,567.55","1,572,173.90","1,632,657.51","68,611.95","1,632,994.40","1,631,407.66"


In [10]:
#There are multiple entries for a cryptocurrency on a given date. Also, the regional MWA data are denominated in their local 
#currency (i.e. nonequivalent units), so you would also need historical exchange rates.

In [45]:
##Step 4: Filter unwanted observations.

#In the previous step, we learned that GWA codes are aggregations of the regional MWA codes.
#Therefore, to perform our analysis, we only need to keep the global GWA codes:

#Number of observations in dataset
print ('Before', len(df))

#Get all the GWA codes
gwa_codes = [code for code in df.Code.unique() if 'GWA_' in code]

#Keep only GWA observations
df = df[df.Code.isin(gwa_codes)]

#Number of observations left
print ('After', len(df))
df.head()

Before 6309
After 6309


,Code,Date,Open,High,Low,Close,Volume,VWAP,TWAP
0,GWA_BTC,2014-04-01,467.28,488.62,467.28,479.56,"74,776.48",482.76,482.82
1,GWA_BTC,2014-04-02,479.20,494.30,431.32,437.08,"114,052.96",460.19,465.93
2,GWA_BTC,2014-04-03,437.33,449.74,414.41,445.60,"91,415.08",432.29,433.28
3,GWA_BTC,2014-04-04,445.18,456.10,429.16,449.81,"51,147.27",443.46,443.93
4,GWA_BTC,2014-04-05,450.08,464.09,445.16,461.70,"28,449.19",452.53,452.95


In [13]:
#Now that we only have GWA codes left, all of our observations are equivalent in granularity and in units. 
#We can confidently proceed.

In [14]:
##Step 5: Pivot the dataset.

#We'll need calculate returns over the prior 7, 14, 21, and 28 days... for the first day of each month.
#Hence, we will pivot the table (Return reshaped DataFrame organized by given index / column values).

pivoted_df = df.pivot(index = 'Date', columns='Code', values='VWAP')
pivoted_df.tail()

Code,GWA_BTC,GWA_ETH,GWA_LTC,GWA_XLM,GWA_XRP
Date,,,,,
2018-01-19,"11,826.36","1,068.45",195.00,0.51,1.82
2018-01-20,"13,062.68","1,158.71",207.58,0.52,1.75
2018-01-21,"12,326.23","1,108.90",197.36,0.48,1.55
2018-01-22,"11,397.52","1,038.21",184.92,0.47,1.43
2018-01-23,"10,921.00",992.05,176.95,0.47,1.42


In [15]:
#As you can see, each column in our pivoted dataset now represents the price for one cryptocurrency and each row contains prices
#from one date. All the features are now aligned by date.

In [16]:
##Step 6: Shift the pivoted dataset.

#To easily calculate returns over the prior 7, 14, 21, and 28 days, we can use Pandas's shift method. 
#This function shifts the index of the dataframe by some number of periods.

#Note: Calculating returns by shifting the dataset requires 2 assumptions to be met: 
#(1) the observations are sorted ascending by date and 
#(2) there are no missing dates. 

delta_dict = {}

for offset in [7, 14, 21, 28]:
    delta_dict['delta_{}'.format(offset)] = pivoted_df / pivoted_df.shift(offset) -1.0

In [19]:
print(delta_dict)

{'delta_7': Code        GWA_BTC  GWA_ETH  GWA_LTC  GWA_XLM  GWA_XRP
Date                                                   
2014-04-01      nan      nan      nan      nan      nan
2014-04-02      nan      nan      nan      nan      nan
2014-04-03      nan      nan      nan      nan      nan
2014-04-04      nan      nan      nan      nan      nan
2014-04-05      nan      nan      nan      nan      nan
2014-04-06      nan      nan      nan      nan      nan
2014-04-07      nan      nan      nan      nan      nan
2014-04-08    -0.06      nan    -0.14      nan    -0.08
2014-04-09    -0.02      nan    -0.06      nan     0.05
2014-04-10    -0.07      nan    -0.04      nan     0.03
2014-04-11    -0.09      nan    -0.07      nan    -0.10
2014-04-12    -0.05      nan     0.01      nan    -0.13
2014-04-13    -0.10      nan    -0.10      nan    -0.22
2014-04-14    -0.01      nan    -0.03      nan    -0.16
2014-04-15     0.07      nan     0.08      nan    -0.09
2014-04-16     0.15      nan     0.1

In [36]:
##Step 7: Melt the shifted dataset.

#Now that we've calculated returns using the pivoted dataset, we're going to "unpivot" the returns.
#By unpivoting, or melting the data, we can later create an analytical base table (ABT) where each row 
#contains all of the relevant information for a particular coin on a particular date.

#Melt all dataframes and store in list
melted_dfs = []
for key, delta_df in delta_dict.items():
    melted_dfs.append(delta_df.reset_index().melt(id_vars=['Date'], value_name=key))
    

In [37]:
#Finally, we can create another melted dataframe that contains the forward-looking 7-day returns.
#This will be our "target variable" for evaluating our trading strategy.

return_df = pivoted_df.shift(-7) / pivoted_df - 1.0
melted_dfs.append(return_df.reset_index().melt(id_vars=['Date'], value_name='return_7'))

In [38]:
#We now have 5 melted dataframes stored in the list, one for each of the backwardlooking
#7, 14, 21, and 28-day returns and one for the forward-looking 7-day returns.

In [39]:
##Step 8: Reduce-merge the melted data.

#All that's left to do is join our melted dataframes into a single analytical base table, using reduce-merge.

#Grab features from original dataset
base_df = df[['Date', 'Code', 'Volume', 'VWAP']]

#Create a list with all the features dataframes
feature_dfs = [base_df] + melted_dfs

#Reduce-Merge features into ABT
from functools import reduce #Reduce applies a function of two arguments cumulatively to the objects in a sequence

abt = reduce(lambda left, right: pd.merge(left, right, on=['Date', 'Code']), feature_dfs)
abt.tail(10)

,Date,Code,Volume,VWAP,delta_7,delta_14,delta_21,delta_28,return_7
6299,2018-01-14,GWA_XRP,"912,107,674.18",2.20,-0.31,0.02,1.06,1.97,-0.30
6300,2018-01-15,GWA_XRP,"823,491,754.55",2.11,-0.29,-0.04,0.92,1.84,-0.32
6301,2018-01-16,GWA_XRP,"3,872,977,355.95",1.48,-0.46,-0.36,0.35,0.85,-0.04
6302,2018-01-17,GWA_XRP,"5,111,390,628.85",1.20,-0.47,-0.57,-0.06,0.58,nan
6303,2018-01-18,GWA_XRP,"5,156,172,462.44",1.68,-0.22,-0.51,0.23,0.59,nan
6304,2018-01-19,GWA_XRP,"2,126,239,927.56",1.82,-0.22,-0.41,0.02,0.65,nan
6305,2018-01-20,GWA_XRP,"1,346,913,296.52",1.75,-0.29,-0.42,-0.26,0.53,nan
6306,2018-01-21,GWA_XRP,"1,886,060,450.81",1.55,-0.30,-0.51,-0.28,0.45,nan
6307,2018-01-22,GWA_XRP,"1,784,992,299.63",1.43,-0.32,-0.52,-0.35,0.30,nan
6308,2018-01-23,GWA_XRP,"2,118,335,564.32",1.42,-0.04,-0.48,-0.39,0.29,nan


In [40]:
#Data Dictionary for our Analytical Base Table (ABT):
#• Date: The day on which the index values were calculated.
#• Code: Which cryptocurrency.
#• VWAP: The volume weighted average price traded that day.
#• delta_7: Return over the prior 7 days (1.0 = 100% return).
#• delta_14: Return over the prior 14 days (1.0 = 100% return).
#• delta_21: Return over the prior 21 days (1.0 = 100% return).
#• delta_28: Return over the prior 28 days (1.0 = 100% return).
#• return_7: Future return over the next 7 days (1.0 = 100% return).

#Notice how the last 7 observations don't have values for the feature? This is expected, as we cannot calculate 
#"future 7-day returns" for the last 7 days of the dataset.

In [41]:
#if we wanted to pick the coin that had the biggest momentum on September 1st, 2017, we could simply display
#the rows for that date and look at the 7, 14, 21, and 28-day prior returns:

#Data from Sept 1st, 2017
abt[abt.Date=='2017-09-01']

,Date,Code,Volume,VWAP,delta_7,delta_14,delta_21,delta_28,return_7
1249,2017-09-01,GWA_BTC,"275,034.79","4,798.06",0.10,0.12,0.35,0.69,-0.09
2149,2017-09-01,GWA_ETH,"2,076,778.42",387.55,0.17,0.28,0.29,0.72,-0.21
3543,2017-09-01,GWA_LTC,"18,553,463.67",78.76,0.55,0.71,0.68,0.82,-0.10
4770,2017-09-01,GWA_XLM,"372,143,342.95",0.02,0.19,0.34,0.05,0.10,-0.19
6164,2017-09-01,GWA_XRP,"1,138,500,431.07",0.25,0.15,0.57,0.39,0.44,-0.14


In [42]:
#if you wanted to programmatically pick the crypto with the biggest momentum (e.g. over the prior 28 days), you would write:

max_momentum_id = abt[abt.Date == '2017-09-01'].delta_28.idxmax()
df.loc[max_momentum_id, ['Code', 'return_7']]

C:\Users\wmiranda\Anaconda3\lib\site-packages\pandas\core\indexing.py:1017: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]


Code        GWA_LTC
return_7        NaN
Name: 3543, dtype: object

In [43]:
##Step 9: (Optional) Aggregate with group-by.

#As a final step, if we wanted to only keep the first days of each month, we can use a group-by followed by an aggregation.

#Create 'month' feature
abt['month'] = abt.Date.apply(lambda x: x[:7])

#Group by code and month and keep the first date
gb_df = abt.groupby(['Code', 'month']).first().reset_index()

#Daily Examples
gb_df.tail()


,Code,month,Date,Volume,VWAP,delta_7,delta_14,delta_21,delta_28,return_7
204,GWA_XRP,2017-09,2017-09-01,"1,138,500,431.07",0.25,0.15,0.57,0.39,0.44,-0.14
205,GWA_XRP,2017-10,2017-10-01,"242,650,661.68",0.20,0.12,0.12,-0.05,-0.13,0.31
206,GWA_XRP,2017-11,2017-11-01,"344,634,241.50",0.20,-0.03,-0.11,-0.25,-0.05,0.07
207,GWA_XRP,2017-12,2017-12-01,"705,807,839.14",0.25,0.03,0.09,0.17,0.19,-0.00
208,GWA_XRP,2018-01,2018-01-01,"813,772,647.20",2.20,1.01,1.97,7.96,7.77,0.35


In [44]:
#As you can see, we now have a proper ABT with:
#• Only relevant data from the 1st day of each month.
#• Momentum features calculated from the prior 7, 14, 21, and 28 days.
#• The future returns you would've made 7 days later.